# Weather time series data set (with PyQrack quantum associative memory)

"PyQrack" is a pure Python language standard wrapper for the (C++11) Qrack quantum computer simulator library. PyQrack exposes a "quantum neuron" called "`QrackNeuron`." (Its API reference is [here](https://pyqrack.readthedocs.io/en/latest/autoapi/pyqrack/qrack_neuron/index.html).) We'd like to model a simple data set to achieve a proof-of-concept of using `QrackNeuron`.

In [1]:
#!pip install pyqrack

First, load the data set into a `pandas` dataframe.

In [2]:
import math
import random
import numpy as np
import pandas as pd

train = pd.read_csv('time-series/DailyDelhiClimateTrain.csv')
train['date'] = pd.to_datetime(train['date'], format='%Y-%m-%d').apply(lambda x:x.toordinal())

test = pd.read_csv('time-series/DailyDelhiClimateTest.csv')
test['date'] = pd.to_datetime(test['date'], format='%Y-%m-%d').apply(lambda x:x.toordinal())

print(train.head())
print("Number of observations: ", train.shape[0])

     date   meantemp   humidity  wind_speed  meanpressure
0  734869  10.000000  84.500000    0.000000   1015.666667
1  734870   7.400000  92.000000    2.980000   1017.800000
2  734871   7.166667  87.000000    4.633333   1018.666667
3  734872   8.666667  71.333333    1.233333   1017.166667
4  734873   6.000000  86.833333    3.700000   1016.500000
Number of observations:  1462


Separate the dependent and independent variables.

In [3]:
# keys = ['Holiday_Flag', 'Store', 'Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
keys = ['date', 'humidity', 'wind_speed', 'meanpressure']
dep_key = 'meantemp'

X = train[keys]
y = train[dep_key]

X_test = test[keys]
y_test = test[dep_key]

Ideally, we'd like to make an improvement on the goodness-of-fit of multiple linear regression, with PyQrack's `QrackNeuron`. At the very least, to show that `QrackNeuron` can be viable for modeling a data set, we'd like to show somehwat comparable performance to multiple linear regression.

PyQrack's `QrackNeuron` can only work with discrete, binary data. To model this or any data set, we have to reduce it to a simple, discrete, binary form.

We'll try to model the data set via "(quantum) associative memory." There are several statistical considerations, to avoid overfit.

Firstly, each possible discretized independent variable permutation input trains an independent parameter of a `QrackNeuron`. If a `QrackNeuron` has never seen a specific, exact permutation of input bits, it has no information about them at all, so its prediction defaults to "maximal superposition," (i.e. a totally random guess). Therefore, we'd like to keep our number of possible distinct inputs significantly fewer in number than our observation rows, when we discretize our indepedent variables.

Satisfying the first consideration, we secondly discretize our dependent variable to have exactly as many possible discrete values as possible distinct inputs. (We guess that this loses the least information about the dependent variable, while we still have enough observations to fully train our network.)

Thirdly, our learning rate should should just barely "saturate" the learned parameters of our (quantum) associative memory. As a learning volatility parameter ("`eta`") of `1/2` "fully trains" one parameter of a `QrackNeuron` between input qubits and output qubit, on average, this implies that we might set `eta` to `1/2` times `2` to the power of input qubits (summed across all predictors) divided by the number of observations. 

At a baseline, our first choice to model this data set might be multiple linear regression.

In [4]:
from sklearn import linear_model

regr = linear_model.LinearRegression()
regr.fit(X, y)
pd.DataFrame(zip(X.columns, regr.coef_))

,0,1
0,date,0.001871
1,humidity,-0.229847
2,wind_speed,0.180727
3,meanpressure,-0.001512


In [5]:
y_pred = regr.predict(X_test)
sst = 0
ssr = 0
for i in range(len(y_pred)):
    sst += y_test[i] * y_test[i]
    ssr += (y_test[i] - y_pred[i]) * (y_test[i] - y_pred[i])

print("Multiple linear regression validation R^2: ", 1 - ssr / sst)

Multiple linear regression validation R^2:  0.8927124450837762


To discretize the data, we split it into as many quantiles as `2` to the power of our number of input qubits.

In [6]:
time_qubit_count = 2
in_qubit_count = 2
out_qubit_count = 3

time_bin_count = 1  << time_qubit_count
in_key_count = len(keys)
in_bin_count = 1 << in_qubit_count
in_tot_count = time_qubit_count + (in_key_count - 1) * in_qubit_count
out_bin_count = 1 << out_qubit_count

x_bins = []
x_bins.append(np.percentile(X[keys[0]], np.arange(0, 100, 100 / time_bin_count)))
for i in range(1, len(keys)):
    key = keys[i]
    x_bins.append(np.percentile(X[key], np.arange(0, 100, 100 / in_bin_count)))
y_bins = np.percentile(y, np.arange(0, 100, 100 / out_bin_count))

In [7]:
print(x_bins)

[array([734869.  , 735234.25, 735599.5 , 735964.75]), array([13.42857143, 50.375     , 62.625     , 72.21875   ]), array([0.        , 3.475     , 6.22166667, 9.23823529]), array([  -3.04166667, 1001.58035714, 1008.56349206, 1014.94490132])]


In [8]:
print(y_bins)

[ 6.         15.5        18.85714286 23.70833333 27.71428571 29.875
 31.30580357 33.125     ]


Once we have our quantiles, we bin our indepedent training and validation data.

In [9]:
def discretize_x(X):
    xd = []
    for i in X.index:
        xd_row = []
        
        key = keys[0]
        bn = time_bin_count
        offset = 0
        while bn > 1:
            bn =  bn // 2
            b = bn + offset
            if X[keys[0]][i] < x_bins[0][b - 1]:
                xd_row.append(False)
            else:
                xd_row.append(True)
                offset += bn

        for ki in range(1, len(keys)):
            key = keys[ki]
            bn = in_bin_count
            offset = 0
            while bn > 1:
                bn =  bn // 2
                b = bn + offset
                if X[key][i] < x_bins[ki][b - 1]:
                    xd_row.append(False)
                else:
                    xd_row.append(True)
                    offset += bn

        xd.append(xd_row)
    return xd

xd = discretize_x(X)
xd_test = discretize_x(X_test)

We do the same for our dependent data.

In [10]:
def discretize_y(y):
    yd = []
    for i in y.index:
        yd_row = []
        bn = out_bin_count
        offset = 0
        while bn > 0:
            bn =  bn // 2
            b = bn + offset
            if y[i] < y_bins[b - 1]:
                yd_row.append(False)
            else:
                yd_row.append(True)
                offset += bn
        yd.append(yd_row)
    return yd

yd = discretize_y(y)
yd_test = discretize_y(y_test)

We're ready to train our associative memory! (Note that it offers us no particular advantage, in this case, that our "neurons" are based on simulated quantum computational gates, though it is possible to predict in "superposition" of many rows at once.)

In [11]:
from IPython.display import clear_output
from pyqrack import QrackSimulator, QrackNeuron

eta = (1 / 2) * (out_bin_count / y.shape[0])
input_indices = list(range(in_tot_count))
qsim = QrackSimulator(in_tot_count + out_qubit_count)

qft_qubits = list(range(time_qubit_count))

output_layer = []
for i in range(out_qubit_count):
    output_layer.append(QrackNeuron(qsim, input_indices, in_tot_count + i))

# Train the network to associate powers of 2 with their log2()
print("Learning...")
for i in range(len(xd)):
    clear_output(wait=True)
    print("Epoch ", (i + 1), " out of ", len(xd))
    
    perm = xd[i]
    res = yd[i]

    for j in range(out_qubit_count):
        qsim.reset_all()
        for k in range(in_tot_count):
            if perm[k]:
                qsim.x(k)
        # Transform time domain to Fourier basis
        qsim.qft(qft_qubits)
        output_layer[j].learn(eta, res[j] == 1)

Epoch  1462  out of  1462


Let's use our neural net, trained on half the data, to try to predict the left-out half of data!

In [12]:
print("Should associate each input with its trained output...")
sum_sqr_tot = 0
sum_sqr_res = 0
for i in range(len(xd_test)):
    clear_output(wait=True)
    print("Predicting ", (i + 1), " out of ", len(xd_test))
    
    perm = xd_test[i]

    qsim.reset_all()
    for j in range(in_tot_count):
        if perm[j]:
            qsim.x(j)
    # Transform time domain to Fourier basis
    qsim.qft(qft_qubits)

    for j in range(out_qubit_count):
        output_layer[j].predict()

    bn = out_bin_count
    offset = 0
    for j in range(out_qubit_count):
        bn = bn // 2
        offset += bn * qsim.prob(in_tot_count + j) 
    pred = y_bins[round(offset)]

    sum_sqr_tot += y_test[i] * y_test[i]
    sum_sqr_res += (y_test[i] - pred) * (y_test[i] - pred)

Predicting  114  out of  114


How does this compare to the validation R^2 of multiple linear regression?

In [13]:
print("QrackNeuron training R^2: ", 1 - sum_sqr_res / sum_sqr_tot)
print("Multiple linear regression training R^2: ", 1 - ssr / sst)
if (1 - sum_sqr_res / sum_sqr_tot) > (1 - ssr / sst):
    print("QrackNeuron quantum associative memory outperformed multiple linear regression.")
else:
    print("Multiple linear regression outperformed QrackNeuron quantum associative memory.")

QrackNeuron training R^2:  0.9475895668699247
Multiple linear regression training R^2:  0.8927124450837762
QrackNeuron quantum associative memory outperformed multiple linear regression.
